In [ ]:
# This notebook consists of tests for SeismicGeometry
# Note that it runs only for one predefined dataformat from available formats
import os
import sys
from datetime import date
import warnings
warnings.filterwarnings("ignore")

import numpy as np

sys.path.append('../../seismiqb')

from seismiqb import SeismicGeometry

In [ ]:
""" You should pay attention to the next parameter:

FORMAT : str
    A file format with SeismicGeometry. Possible options are: 'sgy', 'hdf5', 'qhdf5', 'blosc', 'qblosc'.
    
It is used for running this notebook for all available data formats in the test script.
"""
# Workspace constants
DATESTAMP = date.today().strftime("%Y-%m-%d")
TESTS_ROOT_DIR = './'

# Tests parameters
FORMAT = 'sgy'
N_SLIDE = 1000
N_CROP = 300
SEED = 42
ATTRIBUTES_NAMES = ['snr', 'std_matrix', 'quality_map', 'quality_grid']

# Visualization parameters
SCALE = 1

# Output parameters
VERBOSE = True

In [ ]:
# Path to cube (depends on the data format)
OUTPUT_DIR = os.path.join(TESTS_ROOT_DIR, "geometry_test_files")
CUBE_NAME = f'test_cube_{DATESTAMP}.{FORMAT}'
CUBE_PATH = os.path.join(OUTPUT_DIR, CUBE_NAME)

# Data for tests:
with open(os.path.join(OUTPUT_DIR, f'test_array_{DATESTAMP}.npy'), 'rb') as infile:
    data_array = np.load(infile)

# Tests

In [ ]:
%%time
geometry = SeismicGeometry(CUBE_PATH)

In [ ]:
# Check data
geometry_data = geometry[:, :, :].ravel()

if not geometry.quantized:
    error_msg = "Check data test failed: data is not the same"
    assert (geometry_data == data_array.ravel()).all(), error_msg
else:
    corr = np.corrcoef(geometry_data, data_array.ravel())[0, 1]
    assert corr >=0.9, "Check data test failed: data is not correlated"

In [ ]:
# Attributes loading
geometry.make_quality_map(
    [0.1], ['support_js', 'support_hellinger'], safe_strip=0
) # safe_strip=0 because data is a noise and otherwise haven't good traces

for attr_name in ATTRIBUTES_NAMES:
    attr = geometry.__getattr__(key=attr_name)

    error_msg = f"Attributes loading test failed (the gettattr method for {attr_name})"
    assert attr is not None, error_msg

    attr = geometry.load_attribute(src=attr_name)

    error_msg = f"Attributes loading test failed (the load_attribute method for {attr_name})"
    assert attr is not None, error_msg

    if VERBOSE:
        print(f"Attribute {attr_name} was successfully loaded.")

In [ ]:
# Slide loading
axis = np.random.randint(3)
loc = np.random.randint(geometry.cube_shape[axis])
data_slice = [slice(None) for i in range(axis)]
data_slice.append(loc)

geometry_slide = geometry.load_slide(loc=loc, axis=axis)
data_slide = data_array[tuple(data_slice)]

if not geometry.quantized:
    error_msg = "Slide loading test failed: data is not the same"
    assert (geometry_slide == data_slide).all(), error_msg
else:
    corr = np.corrcoef(geometry_slide.ravel(), data_slide.ravel())[0, 1]
    assert corr >=0.9, "Slide loading test failed: data is not correlated"

In [ ]:
# Crop loading
point = np.random.randint(geometry.cube_shape) // 2
shape = np.random.randint((5, 5, 5), (200, 200, 200))
locations = [slice(start_, np.clip(start_+shape_, 0, max_shape))
             for start_, shape_, max_shape in zip(point, shape, geometry.cube_shape)]

geometry_crop = geometry.load_crop(locations=locations)
data_crop = data_array[tuple(locations)]

if not geometry.quantized:
    error_msg = "Crop loading test failed: data is not the same"
    assert (geometry_crop == data_crop).all(), error_msg
else:
    corr = np.corrcoef(geometry_crop.ravel(), data_crop.ravel())[0, 1]
    assert corr >=0.9, "Crop loading test failed: data is not correlated"

In [ ]:
# Visualizations
axis = np.random.randint(3)
loc = np.random.randint(geometry.cube_shape[axis])

geometry.show_slide(loc=loc, axis=axis, scale=SCALE)
geometry.show_histogram(hist_log=True, scale=SCALE)

geometry.show(matrix='mean_matrix', scale=SCALE)
geometry.show(matrix='snr', scale=SCALE)

geometry.show_quality_map(scale=SCALE)
geometry.show_quality_grid(scale=SCALE)

In [ ]:
# Timings
timings = geometry.benchmark(n_slide=N_SLIDE, n_crop=N_CROP, use_cache=False, seed=SEED)
timings = {FORMAT.upper(): timings}